In [1]:
import pandas as pd
import numpy as np
from progressbar import progressbar as pb
import re

df = pd.read_csv('big_data_fix.csv')

In [2]:
print(len(df))
df.head()

13073


,0,1,2,3,4,5
0,Совещательный орган при императоре в начале XX в,Сенат,Государственный совет,Консилия министров,Верховный тайный совет,2
1,Министр внутренних дел с 1904 г либерал автор ...,Лорис Меликов,Святополк Мирский,Милюков,Витте,2
2,Какое из названных событий произошло 27 января...,подрыв флагманского корабля Петропавловск,высадка японских войск на Ляодунском полуострове,героический бой Варяга и Корейца,заключение Портсмутского мирного договора,3
3,Что из названного относится к результатам перв...,появление многопартийности,формирование конституционной монархии,ликвидация сословного строя,введение рабочего контроля за производством,1
4,Творчество поэтов Гумилева Ахматовой Мандельшт...,акмеизму,футуризму,импрессионизму,реализму,1


In [3]:
embs = np.zeros((df.shape[0], df.shape[1]-1, 300))
embs.shape

(13073, 5, 300)

In [4]:
import gensim
import pymorphy2
import zipfile

model_file = '/Users/user/Python/Diploma/rusvectores.zip'
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [5]:
lst_df = df.values.tolist()
lst_df[1]

['Министр внутренних дел с 1904 г либерал автор проекта О мерах к усовершенствованию государственного порядка',
 'Лорис Меликов',
 'Святополк Мирский',
 'Милюков',
 'Витте',
 2]

In [6]:
all_tokens = []

for i in range(len(lst_df)):
    for j in range(5):
        sent_words = re.findall('[А-Яа-яЁёA-Za-z]+', lst_df[i][j])
        lst_df[i][j] = sent_words
        all_tokens.extend(sent_words)

In [7]:
len(all_tokens)
uniq_tokens = list(set(all_tokens))[1:]
len(uniq_tokens)

56674

In [8]:
lst_df[0]

[['Совещательный', 'орган', 'при', 'императоре', 'в', 'начале', 'XX', 'в'],
 ['Сенат'],
 ['Государственный', 'совет'],
 ['Консилия', 'министров'],
 ['Верховный', 'тайный', 'совет'],
 2]

In [9]:
morph = pymorphy2.MorphAnalyzer()

def prepare_token(token):
    p = morph.parse(token)[0]
    normal_form, pos = p.normal_form, p.tag.POS
    if pos == 'ADJF' or pos == 'ADJS' or pos == 'COMP':
        pos = 'ADJ'
    if pos == 'ADVB':
        pos = 'ADV'
    if pos == 'INFN' or pos == 'PRTF' or pos == 'PRTS' or pos == 'GRND':
        pos = 'VERB'
    if pos == 'CONJ':
        pos = 'CCONJ'
    if pos == 'PREP':
        pos = 'ADP'
    if pos == 'NPRO':
        pos = 'PRON'
    return str(normal_form + '_' + pos)

In [10]:
tokens_dict = {}

In [11]:
for token in pb(uniq_tokens):
    try:
        tokens_dict[token] = model.get_vector(prepare_token(token))
    except:
        continue

100% (56674 of 56674) |##################| Elapsed Time: 0:00:11 Time:  0:00:11


In [12]:
len(tokens_dict)

43287

In [13]:
def get_vec(lst):
    res = np.zeros((5,300))
    for i in range(len(lst)-1):
        cur_vec = np.zeros(300)
        for token in lst[i]:
            try:
                cur_vec += tokens_dict[token]
            except:
                continue
        try:
            res[i,:] = cur_vec / len(cur_tag)
        except:
            res[i,:] = cur_vec
    return res

In [14]:
from multiprocessing import Pool
from tqdm import tqdm

p = Pool(6)

rusvec_embs = p.map(get_vec, lst_df)

In [16]:
rusvec_embs = np.array(rusvec_embs)

import pickle

with open('rusvec_embed_big.pickle', 'wb') as f:
    pickle.dump(rusvec_embs, f)